<h1>1.读取认证信息和租户信息</h1>
认证信息不需要在每次调用接口时都去通过登录重新获取，认证信息的有效时间是7天，因此在7天内都可以重复使用，可以直接读取user_info.json中保存的认证信息。

In [ ]:
import json

with open('user_info.json', 'r', encoding='utf-8') as f:
    payload = json.load(f)

token=payload["token"]
tenantId=payload["tenantId"]
print("认证信息：",token)
print("租户信息：",tenantId)

<h1>opc ua接入实测数据有两种方式，一种是直接获取到工控设备的当前实时数据。另一种是将所有的设备数据同步到iot服务后，使用统一接口读取同步过来的数据</h1>

<h1>2.推送数据(opc-ua)：方式一</h1>
<h2>第一步:初始化opc ua 服务</h2>
初始化时需要填写opc ua server信息，包括opcFlag、opcServerIp、opcServerPort、userName、password、remarks。</br>
opcFlag：opc ua服务器的标识，后续在获取工控数据、同步数据到iot服务时都会用到。</br>
opcServerIp、opcServerPort、userName、password：分别是opc ua服务器的ip、端口号、登录名、密码等数据。</br>
remarks：备注信息，可空

In [ ]:
import os
from dhicn_iot_service import *

configuration=Configuration.get_default_copy()
configuration.host="https://online-products.dhichina.cn/gateway/iot-service" #服务地址
api_client=ApiClient(configuration)
api_client.set_default_header('Authorization',token)
api_client.set_default_header('tenantId',tenantId)

opcua_instance=OpcuaApi(api_client)
init_opc_ua_config_input=[{
        "opcFlag": "opcua_test_1",
        "opcServerIp": "172.16.176.36",
        "opcServerPort": "34561",
        "userName": 'admin',
        "password": '123456',
        "remarks": ''
}]
response=opcua_instance.api_v1_opc_ua_init_post(init_opc_ua_config_input=init_opc_ua_config_input)

print("result:", response)
if response["status"] == False:
    os._exit()

<h2>第二步：添加opc ua 监测点</h2>

In [ ]:
import os
from dhicn_iot_service import *
def find_key(key, array):
    if isinstance(array, dict):
        if array['key']==key:
            return array['value']
        else:
             return None
    elif isinstance(array, list):
        for item in array:
            result = find_key(key, item)
            if result is not None:
                return result


configuration=Configuration.get_default_copy()
configuration.host="https://online-products.dhichina.cn/gateway/iot-service" #服务地址
api_client=ApiClient(configuration)
api_client.set_default_header('Authorization',token)
api_client.set_default_header('tenantId',tenantId)

#获取设备信息
device_instance=DeviceApi(api_client)
param={
    'page_index':1,
    'page_size':1000
}
response = device_instance.api_v1_iot_device_list_get(**param)
attributes=response["data"]["list"][0]["attributes"]
indicators = find_key("indicators",attributes)
print(indicators)
indicators=json.loads(indicators)
print(indicators[0]["Indicator"],indicators[0]["Id"])
print(indicators[1]["Indicator"],indicators[1]["Id"])

#构建参数
opcua_instance=OpcuaApi(api_client)
add_opc_ua_com_input=[{
    "indicatorId": indicators[0]["Id"],
    "nodeName": "流量",
    "node": "ns=2;s=Devices/测试节点/测试设备/流量",
    "unit": "m³/s",
    "dataType": 2,
    "decimalPlaces": 0,
    "timeProcessingMode": 0,
    "period": 0,
    "opcFlag": "opcua_test_1"
},{
 "indicatorId": indicators[1]["Id"],
    "nodeName": "水位",
    "node": "ns=2;s=Devices/测试节点/测试设备/水位",
    "unit": "m",
    "dataType": 2,
    "decimalPlaces": 0,
    "timeProcessingMode": 0,
    "period": 0,
    "opcFlag": "opcua_test_1"
}]
response=opcua_instance.api_v1_opc_ua_add_communication_post(add_opc_ua_com_input=add_opc_ua_com_input)

print("result:", response)
if response["status"] == False:
    os._exit()

<h2>第三步：读取opc ua 所有监测点</h2>

In [ ]:
import os
from dhicn_iot_service import *
def find_key(key, array):
    if isinstance(array, dict):
        if array['key']==key:
            return array['value']
        else:
             return None
    elif isinstance(array, list):
        for item in array:
            result = find_key(key, item)
            if result is not None:
                return result


configuration=Configuration.get_default_copy()
configuration.host="https://online-products.dhichina.cn/gateway/iot-service" #服务地址
api_client=ApiClient(configuration)
api_client.set_default_header('Authorization',token)
api_client.set_default_header('tenantId',tenantId)

#获取设备信息
device_instance=DeviceApi(api_client)
param={
    'page_index':1,
    'page_size':1000
}
response = device_instance.api_v1_iot_device_list_get(**param)
attributes=response["data"]["list"][0]["attributes"]
indicators = find_key("indicators",attributes)
print(indicators)
indicators=json.loads(indicators)
print(indicators[0]["Indicator"],indicators[0]["Id"])
print(indicators[1]["Indicator"],indicators[1]["Id"])

#构建参数
opcua_instance=OpcuaApi(api_client)
param={}
response=opcua_instance.api_v1_opc_ua_query_communication_post(query_opc_ua_com_input=param)

print("result:", response)
if response["status"] == False:
    os._exit()

<h2>第四步：读取工控实时数据</h2>

In [ ]:

from dhicn_iot_service import *
import os

configuration=Configuration.get_default_copy()
configuration.host="https://online-products.dhichina.cn/gateway/iot-service" #服务地址
api_client=ApiClient(configuration)
api_client.set_default_header('Authorization',token)
api_client.set_default_header('tenantId',tenantId)

opcua_instance=OpcuaApi(api_client)
read_node_input={
    "dataType": "DblValue",
    "node": "ns=2;s=Devices/测试节点/测试设备/流量",
    "opcFlag": "opcua_test_1"
}
param={
    "read_node_input":read_node_input
}
response = opcua_instance.api_v1_opc_ua_read_node_post(**param)

print("result:", response)
if response["status"] == False:
    os._exit()

<h2>3.推送数据(opc-ua)：方式二</h2>

<h2>第一步:初始化opc ua 服务 </h2>
<h2>第二步：添加opc ua 监测点</h2>
<h2>第三步：读取opc ua 所有监测点</h2>
以上第一二三步骤同方式一一致

<h2>第四步：同步工控数据到iot服务</h2>

In [ ]:

from dhicn_iot_service import *
import os

configuration=Configuration.get_default_copy()
configuration.host="https://online-products.dhichina.cn/gateway/iot-service" #服务地址
api_client=ApiClient(configuration)
api_client.set_default_header('Authorization',token)
api_client.set_default_header('tenantId',tenantId)

opcua_instance=OpcuaApi(api_client)

response = opcua_instance.api_v1_opc_ua_sync_telemetry_data_post(tenant_id=tenantId)

print("result:", response)
if response["status"] == False:
    os._exit()

<h2>第五步：获取实测数据</h2>
推送实测数据有三种方式，包括http、opcua、mqtt，分别对应了不同的推送方式，但获取数据的接口都是同一个。也就是如下所示接口</br>
在获取数据时需要用到设备ID/指标名称，因此需要先获取设备信息，解析出deviceId和Indicator。

In [ ]:
from dhicn_iot_service import *

def find_key(key, array):
    if isinstance(array, dict):
        if array['key']==key:
            return array['value']
        else:
             return None
    elif isinstance(array, list):
        for item in array:
            result = find_key(key, item)
            if result is not None:
                return result


configuration=Configuration.get_default_copy()
configuration.host="https://online-products.dhichina.cn/gateway/iot-service" #服务地址
api_client=ApiClient(configuration)
api_client.set_default_header('Authorization',token)
api_client.set_default_header('tenantId',tenantId)

#获取设备信息
device_instance=DeviceApi(api_client)
param={
    'page_index':1,
    'page_size':1000
}
response = device_instance.api_v1_iot_device_list_get(**param)
attributes=response["data"]["list"][0]["attributes"]
indicators = find_key("indicators",attributes)

print(indicators)
indicators=json.loads(indicators)
print(indicators[0]["Indicator"])
print(indicators[1]["Indicator"])

#构建参数
timeseries_input=[{
        "deviceId": response["data"]["list"][0]["id"],
        "endTs": "2023-08-04T18:13:27",
        "keys": [indicators[0]["Indicator"],indicators[0]["Indicator"]],
        "startTs": "2023-08-03T16:13:27",
        "orderBy": "asc"
    },
    {
        "deviceId": response["data"]["list"][1]["id"],
        "endTs": "2023-08-04T18:13:27",
        "keys": [indicators[0]["Indicator"],indicators[0]["Indicator"]],
        "startTs": "2023-08-03T16:13:27",
        "orderBy": "asc"
    }
]
telemetry_instance=TelemetryApi(api_client)
response=telemetry_instance.api_v2_iot_timeseries_batch_post(timeseries_input=timeseries_input)
print("result:", response)
if response["status"] == False:
    os._exit()